In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_pickle('microstucture.pkl')
df.head()

,index,curve_n,curve_area,sizes,ratios,roundness,damage,shrinkage,stiffness_init,stiffness_end,stiffness_ratio
0,0,"[1, 3, 54]","[44.980112526027035, 47.16146038289884, 249.58...","{16: [[11.634028130284323, 5.182346340744194]]...","{16: [2.2449345075253415], 8: [1.6455719360013...","{16: [0.6858273932924801], 8: [0.7353582880135...",0.204720,-0.000794,27534.98,13605.44,0.494115
1,1,"[4, 8, 0]","[278.0840614361895, 144.40479803053336, 0.0]","{16: [[10.922469597792166, 7.119867552525796],...","{16: [1.534083256073687, 1.3558929490918499, 1...","{16: [0.6918901640856667, 0.6835016016191318, ...",0.221336,-0.000548,30629.28,13322.23,0.434951
2,2,"[2, 13, 0]","[125.1913778438531, 215.9813556447589, 0.0]","{16: [[11.30634428058544, 8.841091265113812], ...","{16: [1.2788403537015056, 1.1453809717470627],...","{16: [0.8122733416691932, 0.7154878314704103],...",0.197751,-0.000681,28962.97,13278.01,0.458448
3,3,"[3, 9, 29]","[147.8599102513112, 162.24778265410401, 130.09...","{16: [[8.744291518313723, 3.9869423477763792],...","{16: [2.1932324963742302, 1.872639248932646, 1...","{16: [0.6374008046284033, 0.595356037624701, 0...",0.244860,-0.000608,29644.40,12190.48,0.411224
4,4,"[4, 4, 34]","[214.29927185031744, 67.99614832644124, 148.78...","{16: [[8.53137439993541, 7.901661968749836], [...","{16: [1.0796936687086862, 1.7187243032645076, ...","{16: [0.7162784809176268, 0.6393809837372455, ...",0.232117,-0.000644,29301.05,12944.98,0.441792


In [8]:
df = df.drop(index=5000)
df = df.reset_index()

In [9]:
damage = []
shrinkage = []
stiffness = []

label_path = "reduxed_results/damage_fields/"

for i in range(10000):
    if i != 5000:
        dam = np.load(label_path + f'{i}_110.npy').sum() / (99*99)
        shr = pd.read_csv(label_path + f'stiffness_{i}.csv', sep='\t', usecols=['#shr_observed[-]']).values.tolist()
        stf = pd.read_csv(label_path + f'stiffness_{i}.csv', sep='\t', usecols=['#stiffness[MPa]']).values.tolist()
        damage.append(dam)
        shrinkage.append(shr[-1][0])
        stiffness.append(stf[0] + stf[-1])

In [4]:
df['mortar_damage'] = np.array([df['damage'][i]*32*32/(32*32-sum(df['curve_area'][i])) for i in range(9999)])

In [10]:
df['damage'] = damage
df['shrinkage'] = shrinkage
df['stiffness'] = stiffness

In [11]:
df.head()

,index,curve_n,curve_area,sizes,ratios,roundness,damage,shrinkage,stiffness
0,0,"[1, 3, 54]","[44.980112526027035, 47.16146038289884, 249.58...","{16: [[11.634028130284323, 5.182346340744194]]...","{16: [2.2449345075253415], 8: [1.6455719360013...","{16: [0.6858273932924801], 8: [0.7353582880135...",0.204720,-0.000794,"[27534.98, 13605.44]"
1,1,"[4, 8, 0]","[278.0840614361895, 144.40479803053336, 0.0]","{16: [[10.922469597792166, 7.119867552525796],...","{16: [1.534083256073687, 1.3558929490918499, 1...","{16: [0.6918901640856667, 0.6835016016191318, ...",0.221336,-0.000548,"[30629.28, 13322.23]"
2,2,"[2, 13, 0]","[125.1913778438531, 215.9813556447589, 0.0]","{16: [[11.30634428058544, 8.841091265113812], ...","{16: [1.2788403537015056, 1.1453809717470627],...","{16: [0.8122733416691932, 0.7154878314704103],...",0.197751,-0.000681,"[28962.97, 13278.01]"
3,3,"[3, 9, 29]","[147.8599102513112, 162.24778265410401, 130.09...","{16: [[8.744291518313723, 3.9869423477763792],...","{16: [2.1932324963742302, 1.872639248932646, 1...","{16: [0.6374008046284033, 0.595356037624701, 0...",0.244860,-0.000608,"[29644.4, 12190.48]"
4,4,"[4, 4, 34]","[214.29927185031744, 67.99614832644124, 148.78...","{16: [[8.53137439993541, 7.901661968749836], [...","{16: [1.0796936687086862, 1.7187243032645076, ...","{16: [0.7162784809176268, 0.6393809837372455, ...",0.232117,-0.000644,"[29301.05, 12944.98]"


In [5]:
df.to_pickle('microstucture.pkl')

In [3]:
stiffness_red = np.array([(dam[1] / dam[0]) for dam in df['stiffness']])

In [4]:
df['stiffness_init'] = np.array([dam[0] for dam in df['stiffness']])
df['stiffness_end'] = np.array([dam[1] for dam in df['stiffness']])
df['stiffness_ratio'] = np.array([(dam[1] / dam[0]) for dam in df['stiffness']])

In [13]:
df = df.drop(columns='stiffness')